In [2]:
import pandas as pd
from platform import python_version

from random import sample
from random import seed
print("python version:",python_version())

python version: 3.6.10


In [4]:
# variables that fully define the a/b test results
t = 20   #total number of subjexts(a+b)
a = 10
b = 10
a_yes = 7  #yes count in group a
b_yes = 4  #yes count in group b

#compute remaining a/b test results
t_yes = a_yes+b_yes   #total yes count
t_no = t-t_yes        #total no count
a_yes_pc = 100*a_yes/a    #yes percentage in a
b_yes_pc = 100* b_yes/b   # yes percentage in b

#a/b test statistic : yes percentage difference (a-b)
ab_yes_pc = a_yes_pc - b_yes_pc

print('Observed yes rate(%): A:',a_yes_pc,',b:',b_yes_pc,'a-b:' , ab_yes_pc,'\nTotal counts: Yes:',t_yes,', no:', t_no)


Observed yes rate(%): A: 70.0 ,b: 40.0 a-b: 30.0 
Total counts: Yes: 11 , no: 9


In [10]:
seed(2)  #for reproducible results
bag1 = [1]*t_yes + [0]*t_no   #s1:all results bag
bag2 = sample(bag1, len(bag1)) #shuffle bag
a_rs = bag2[:a]  # S3:random sample a
b_rs = bag2[b:]  #S4: random sample b

#compute the test atatistic
a_yes_pc_rs = 100 *sum(a_rs) / a
b_yes_pc_rs = 100 * sum(b_rs) / b
ab_yes_pc_rs = a_yes_pc_rs - b_yes_pc_rs

print('(1) Bag  :', str(bag1).replace(',', ''),
      '\n(2)Bag shuffled:' , str(bag2).replace(",",""),
      '\n(3)A resample :' , str(a_rs).replace(',',''))

(1) Bag  : [1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0] 
(2)Bag shuffled: [1 1 0 0 1 0 0 1 1 0 1 1 1 0 0 0 1 0 1 1] 
(3)A resample : [1 1 0 0 1 0 0 1 1 0]


In [11]:
seed(2) #for reproducible results
bag = [1]*t_yes + [0]*t_no  #all results bag

#repeat the permutations results for a large number of times
p=100   #number of permutations
perm_res = [0]*p #list for permutation results
for i in range(p):
    bag = sample(bag, k = len(bag))    #shuffle the bag
    a_rs = bag[:a]
    b_rs = bag[a:]
    #compute the test atatistic
    perm_res[i] = 100*sum(a_rs)/a - 100*sum(b_rs)/b
    
#print results
n=15
for i in range(0, len(perm_res),n):
    print(str(perm_res[i:i+n]).replace(",","").replace(".0",""))

[-10 10 -10 10 -30 -10 -10 -10 -10 30 -30 10 10 10 -50]
[-30 10 30 10 -30 10 10 -10 -10 50 -10 -30 -30 10 -10]
[-30 10 10 -10 -30 30 10 -10 30 -10 -10 -10 10 -30 30]
[-50 -30 -10 -10 30 30 30 -10 10 -10 -10 10 -30 10 10]
[10 -10 -30 -10 30 -30 -10 10 10 10 -10 30 50 10 10]
[10 10 -30 -10 -30 30 10 -10 -10 -10 -10 30 -10 -10 10]
[10 -30 30 10 30 30 -30 10 10 -30]


In [12]:
#create the table
perm_res_s = pd.Series(perm_res)
perm_res_s

0    -10.0
1     10.0
2    -10.0
3     10.0
4    -30.0
      ... 
95    30.0
96   -30.0
97    10.0
98    10.0
99   -30.0
Length: 100, dtype: float64

In [13]:
print(pd.pivot_table(perm_res_s.value_counts().reset_index(),
                    values = 0, columns = 'index').to_string(index = False))

 -50.0  -30.0  -10.0   10.0   30.0   50.0
     2     18     31     32     15      2


In [15]:
#two_way hypothesis test( null:a = b, alternative:a!=b)
extreme_count = sum(perm_res_s.abs()>=abs(ab_yes_pc))

# one_way hypothesis test(null: a<=b, alternative: a>b)
pos_extreme_count = sum(perm_res_s >= ab_yes_pc)

print("Number of permutations:",p,
     "\n Two_way: Extreme count:", extreme_count,
     '\n Two_way: Extreme ratio(p-value):' ,extreme_count/p,
     "\n One_way: Extreme count:", pos_extreme_count,
     '\n One_way: Extreme ratio(p-value):' ,pos_extreme_count/p)

Number of permutations: 100 
 Two_way: Extreme count: 37 
 Two_way: Extreme ratio(p-value): 0.37 
 One_way: Extreme count: 17 
 One_way: Extreme ratio(p-value): 0.17
